In [4]:
!pip install mlflow

  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached waitress-3.0.2-py3-none-any.whl.metadata (5.8 kB)
     ---------------------------------------- 0.0/40.0 kB ? eta -:--:--
     -------------------------------------- 40.0/40.0 kB 634.0 kB/s eta 0:00:00
  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached importlib_metadata-8.7.0-py3-none-any.whl.metadata (4.8 kB)
     ---------------------------------------- 0.0/89.9 kB ? eta -:--:--
     ------------------ --------------------- 41.0/89.9 kB ? eta -:--:--
     ------------------------------------ --- 81.9/89.9 kB 1.5 MB/s eta 0:00:01
     ------------------------------------ --- 81.9/89.9 kB 1.5 MB/s eta 0:00:01
     -------------------------------------- 89.9/89.9 kB 564.7 kB/s eta 0:00:00
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.svm import SVR

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

import mlflow

In [10]:
! pip install -q dagshub


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import dagshub
dagshub.init(repo_owner='pankajireo74', repo_name='uber-demand-prediction', mlflow=True)

Initialized MLflow to track repo "pankajireo74/uber-demand-prediction"

Repository pankajireo74/uber-demand-prediction initialized!

In [13]:
# set the dagshub tracking server

mlflow.set_tracking_uri("https://dagshub.com/pankajireo74/uber-demand-prediction.mlflow")

In [14]:
# load the training and test data

train_data_path = "../data/processed/train.csv"
test_data_path = "../data/processed/test.csv"

train_df = pd.read_csv(train_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

test_df = pd.read_csv(test_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

train_df

,lag_1,lag_2,lag_3,lag_4,region,total_pickups,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,187,184.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,194,193.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,180,181.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,197,195.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185,186.0,4
...,...,...,...,...,...,...,...,...
2016-02-29 22:45:00,15.0,9.0,11.0,11.0,29,12,12.0,0
2016-02-29 23:00:00,12.0,15.0,9.0,11.0,29,17,17.0,0
2016-02-29 23:15:00,17.0,12.0,15.0,9.0,29,15,15.0,0


In [15]:
# missing value in training data

train_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

In [16]:
# missing values in the test data

test_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

In [17]:
# make X_train and y_train

X_train = train_df.drop(columns=["total_pickups"])

y_train = train_df["total_pickups"]

In [18]:
X_train.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,184.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,193.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,181.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,195.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,186.0,4


In [19]:
# make X_test and y_test

X_test = test_df.drop(columns=["total_pickups"])

y_test = test_df["total_pickups"]

In [20]:
X_test.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-03-01 00:00:00,36.0,44.0,31.0,29.0,0,41.0,1
2016-03-01 00:15:00,41.0,36.0,44.0,31.0,0,36.0,1
2016-03-01 00:30:00,35.0,41.0,36.0,44.0,0,46.0,1
2016-03-01 00:45:00,47.0,35.0,41.0,36.0,0,35.0,1
2016-03-01 01:00:00,34.0,47.0,35.0,41.0,0,31.0,1


In [21]:
from sklearn import set_config

set_config(transform_output="pandas")

In [22]:
# encode the data

encoder = ColumnTransformer([
    ("ohe", OneHotEncoder(drop="first",sparse_output=False), ["region","day_of_week"])
], remainder="passthrough", n_jobs=-1,force_int_remainder_cols=False)

In [23]:
encoder

,transformers,"[('ohe', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,-1
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,False
,categories,'auto'
,drop,'first'
,sparse_output,False


In [24]:
# encode the train and test data

X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

c:\Users\admin\Documents\DSMP\projects\uber demand predication\uber_demand_predication\venv\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(


In [27]:
import optuna
import tqdm 

c:\Users\admin\Documents\DSMP\projects\uber demand predication\uber_demand_predication\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
# set the experiment

mlflow.set_experiment("Model Selection")

2025/11/13 21:34:42 INFO mlflow.tracking.fluent: Experiment with name 'Model Selection' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/0186045cf1844dafacf21cc18a0d1910', creation_time=1763049884512, experiment_id='0', last_update_time=1763049884512, lifecycle_stage='active', name='Model Selection', tags={}>

In [57]:
def objective(trial):
    # start the child run
    with mlflow.start_run(nested=True) as child:
        
        # model name search space
        list_of_models = ["LR", "RF", "GBR", "XGBR"]
        model_name = trial.suggest_categorical("model_name", list_of_models)
    
        if model_name == "LR":
            model = LinearRegression()
    
        elif model_name == "RF":
            n_estimators_rf = trial.suggest_int("n_estimators_rf",10,100,step=10)
            max_depth_rf = trial.suggest_int("max_depth_rf",3,10)
            model = RandomForestRegressor(n_estimators=n_estimators_rf, 
                                          max_depth=max_depth_rf, 
                                          random_state=42, n_jobs=-1)
    
        elif model_name == "GBR":
            n_estimators_gb = trial.suggest_int("n_estimators_gb",10,100,step=10)
            learning_rate_gb = trial.suggest_float("learning_rate_gb",1e-4,1e-1, log=True)
            model = GradientBoostingRegressor(n_estimators=n_estimators_gb, 
                                              learning_rate=learning_rate_gb,
                                             random_state=42)
    
        elif model_name == "XGBR":
            n_estimators_xgb = trial.suggest_int("n_estimators_xgb",10,100,step=10)
            learning_rate_xgb = trial.suggest_float("learning_rate_xgb",1e-4,1e-1, log=True)
            max_depth_xgb = trial.suggest_int("max_depth_xgb",3,10)
            model = XGBRegressor(n_estimators=n_estimators_xgb,
                                learning_rate=learning_rate_xgb,
                                max_depth=max_depth_xgb)
    
        # log the model name
        mlflow.log_param("model_name",model_name)
        
        # log the model parameters
        mlflow.log_params(model.get_params())
        
        # fit on the data
        model.fit(X_train_encoded,y_train)
    
        # get the predictions
        y_pred = model.predict(X_test_encoded)
    
        # calculate the loss
        loss = mean_absolute_percentage_error(y_test, y_pred)
    
        # log the metric
        mlflow.log_metric("MAPE",loss)
        return loss

In [58]:
# optimize the objective function

with mlflow.start_run(run_name="best_model", nested=True) as parent:

    # create a study object
    study = optuna.create_study(study_name="model_selection", direction="minimize")
    # optimize the objective function
    study.optimize(func=objective, n_trials=50, n_jobs=-1)
    
    # log the best parameters
    mlflow.log_params(study.best_params)
    # log the best error value
    mlflow.log_metric("Best_MAPE", study.best_value)

[I 2025-11-14 00:23:10,868] A new study created in memory with name: model_selection


🏃 View run rumbling-wolf-521 at: https://dagshub.com/pankajireo74/uber-demand-prediction.mlflow/#/experiments/0/runs/f6b1a580e2944c8982a1aba1312ee61c
🧪 View experiment at: https://dagshub.com/pankajireo74/uber-demand-prediction.mlflow/#/experiments/0


[I 2025-11-14 00:23:42,759] Trial 1 finished with value: 0.05817838740110484 and parameters: {'model_name': 'RF', 'n_estimators_rf': 30, 'max_depth_rf': 6}. Best is trial 1 with value: 0.05817838740110484.


🏃 View run omniscient-carp-919 at: https://dagshub.com/pankajireo74/uber-demand-prediction.mlflow/#/experiments/0/runs/bbe0d94cac9045868b51640e72870a57
🧪 View experiment at: https://dagshub.com/pankajireo74/uber-demand-prediction.mlflow/#/experiments/0


[I 2025-11-14 00:23:49,121] Trial 3 finished with value: 0.01332392879357394 and parameters: {'model_name': 'LR'}. Best is trial 3 with value: 0.01332392879357394.


...

🏃 View run stylish-goose-423 at: https://dagshub.com/pankajireo74/uber-demand-prediction.mlflow/#/experiments/0/runs/ee7b29c65c43475ca0fa480a903da0a6
🧪 View experiment at: https://dagshub.com/pankajireo74/uber-demand-prediction.mlflow/#/experiments/0


[I 2025-11-14 00:28:01,032] Trial 42 finished with value: 0.01332392879357394 and parameters: {'model_name': 'LR'}. Best is trial 45 with value: 0.009230211245231115.


🏃 View run stately-auk-624 at: https://dagshub.com/pankajireo74/uber-demand-prediction.mlflow/#/experiments/0/runs/d99028fe71ef421aa7d53d6eabc31477
🧪 View experiment at: https://dagshub.com/pankajireo74/uber-demand-prediction.mlflow/#/experiments/0


[I 2025-11-14 00:29:01,232] Trial 49 finished with value: 0.03679346161114973 and parameters: {'model_name': 'GBR', 'n_estimators_gb': 100, 'learning_rate_gb': 0.0875164890396093}. Best is trial 45 with value: 0.009230211245231115.


🏃 View run best_model at: https://dagshub.com/pankajireo74/uber-demand-prediction.mlflow/#/experiments/0/runs/03b1950afbb54929ab6b0c19381e5aae
🧪 View experiment at: https://dagshub.com/pankajireo74/uber-demand-prediction.mlflow/#/experiments/0


In [59]:
# best value

study.best_value

0.009230211245231115

In [60]:
# best parameters

study.best_params

{'model_name': 'RF', 'n_estimators_rf': 20, 'max_depth_rf': 10}